In [1]:
import string 
import numpy as np 
from gensim.models import Word2Vec
from text_preprocess import NepaliPreprocess
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-03-11 17:28:15.217563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-11 17:28:15.217594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = load_model("LSTM.h5")

2022-03-11 17:28:18.066852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-11 17:28:18.066908: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-11 17:28:18.066942: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (linus-Inspiron-3543): /proc/driver/nvidia/version does not exist
2022-03-11 17:28:18.067351: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-11 17:28:18.092482: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2319688

In [3]:
model_W2V = Word2Vec.load("./pretrained_word2vec_model/nepaliW2V_5Million.model")
nlp = NepaliPreprocess()

In [4]:
def word2token(word):
    try:
        return model_W2V.wv.key_to_index[word]
    except KeyError:
        return 0  
    
# COVERT TOKEN INTO WORD
def token2word(token):
    return model_W2V.wv.index_to_key[token]

In [5]:
translator = str.maketrans('', '', string.punctuation + '–')
def prepare_input(text):

    preprocessed_Text = nlp.Reg_and_Stemming(text)

    translator = str.maketrans('', '', string.punctuation + '–')
    news = preprocessed_Text.translate(translator)
    words = np.array([word2token(w) for w in news.split(' ')[:20] if w != ''])
    input_tokens = []
    for idx in words:
        input_tokens.append(idx) 

    set_x = pad_sequences([input_tokens], maxlen=20, padding='pre', value=0)
    return set_x

In [6]:
def get_CATEGORY(input_Seq):

    opt = model.predict(input_Seq)
    idx = np.argmax(opt)
    category = {'खेलकुद': 0, 'मनोरन्जन': 1, 'व्यापार': 2}
    News_class = list(category.keys())[list(category.values()).index(idx)]
    return News_class

In [13]:
समाचार_शीर्षक = "नेपालले जितका लागि २३३ रनको लक्ष्य पायो"
padded_input = prepare_input(समाचार_शीर्षक)
resut = get_CATEGORY(padded_input)
print('समाचारको प्रकार:', resut)

समाचारको प्रकार: खेलकुद


In [14]:
समाचार_शीर्षक = "फिल्मको बाँकी छायांकनका लागि ढाका उड्यो 'ए मेरो हजुर ४' टिम"
padded_input = prepare_input(समाचार_शीर्षक)
resut = get_CATEGORY(padded_input)
print('समाचारको प्रकार:', resut)

समाचारको प्रकार: मनोरन्जन


In [15]:
समाचार_शीर्षक = "आयल निगमको कार्यकारी निर्देशकमा थानी नियुक्त"
padded_input = prepare_input(समाचार_शीर्षक)
resut = get_CATEGORY(padded_input)
print('समाचारको प्रकार:', resut)

समाचारको प्रकार: व्यापार
